# 4. 並行鏈 (Chains Parallel) - Gemini 版本

本範例展示如何同時執行多個分支，如產品優缺點分析，使用 RunnableParallel 實現並行處理。

## 學習重點
- 使用 RunnableParallel 實現並行處理
- 學習如何同時執行多個分析任務
- 理解並行處理的優勢
- 掌握複雜鏈的組合技巧


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Gemini 模型
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [ ]:
# 定義主要的提示模板 - 產品特徵分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的產品評論家。"),
        ("human", "請列出產品 {product_name} 的主要特徵。"),
    ]
)


In [ ]:
# 定義優點分析函數
def analyze_pros(features):
    """分析產品特徵的優點"""
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的產品評論家。"),
            ("human", "根據這些特徵：{features}，請列出這些特徵的優點。"),
        ]
    )
    return pros_template.format_prompt(features=features)

# 定義缺點分析函數
def analyze_cons(features):
    """分析產品特徵的缺點"""
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的產品評論家。"),
            ("human", "根據這些特徵：{features}，請列出這些特徵的缺點。"),
        ]
    )
    return cons_template.format_prompt(features=features)

print("分析函數已定義：")
print("1. analyze_pros: 分析產品優點")
print("2. analyze_cons: 分析產品缺點")


In [ ]:
# 定義結果合併函數
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"優點分析：\n{pros}\n\n缺點分析：\n{cons}"

print("結果合併函數已定義：combine_pros_cons")


In [ ]:
# 使用 LCEL 簡化分支鏈的建立
# 優點分析分支
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

# 缺點分析分支
cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

print("並行分支鏈已建立：")
print("1. pros_branch_chain: 優點分析分支")
print("2. cons_branch_chain: 缺點分析分支")


In [ ]:
# 建立完整的並行處理鏈
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(
        branches={
            "pros": pros_branch_chain, 
            "cons": cons_branch_chain
        }
    )
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

print("並行處理鏈已建立完成！")
print("鏈的流程：產品特徵 → 並行分析(優點+缺點) → 合併結果")


In [ ]:
# 執行並行處理鏈
result = chain.invoke({"product_name": "MacBook Pro"})

# 輸出結果
print("=" * 60)
print("產品分析結果：")
print("=" * 60)
print(result)


## 💡 重點說明

### 並行處理的優勢

1. **效率提升**: 同時執行多個分析任務，節省時間
2. **資源利用**: 充分利用系統資源，提高處理速度
3. **模組化**: 每個分支都是獨立的，易於維護和修改

### RunnableParallel 的使用

```python
# 基本語法
RunnableParallel(
    branches={
        "分支名稱1": 分支鏈1,
        "分支名稱2": 分支鏈2,
        # ... 更多分支
    }
)
```

### 並行鏈的資料流

1. **輸入**: 產品名稱
2. **特徵分析**: 生成產品特徵列表
3. **並行處理**: 同時分析優點和缺點
4. **結果合併**: 將並行結果整合成最終報告

### 實際應用場景

- 產品評論分析
- 多角度文本分析
- 並行數據處理
- 多任務同時執行
